In [1]:
import pandas as pd
import numpy as np
import docplex
from docplex.mp.model import Model
from docplex.mp.progress import *
import time
import sys
import math

In [2]:
def DataPreperation(datasetname,sampleAmount, featCART):
    
    global data
    global trainData
    global samplePos
    global trainDataDummies
    global groupsOffeatures
    global featureToGroup
    global numericalGroup
    numericalGroup = []
    # Read in prefered dataset
    if(datasetname == 'chess'):
        dataChess = pd.read_csv("kr-vs-kp.data", header=0, names=['bkblk','bknwy','bkon8','bkona','bkspr','bkxbq','bkxcr','bkxwp','blxwp','bxqsq','cntxt','dsopp','dwipd','hdchk','katri','mulch','qxmsq','r2ar8','reskd','reskr','rimmx','rkxwp','rxmsq','simpl','skach','skewr','skrxp','spcop','stlmt','thrsk','wkcti','wkna8','wknck','wkovl','wkpos','wtoeg','white_win'])
        if featCART:
            dataChess = dataChess[['white_win','rimmx','wknck','bxqsq','wkna8', 'wkpos','bkxbq','katri', 'bkblk']]
        data = dataChess.dropna().sample(n=sampleAmount, random_state=1)
        trainData = data.drop(columns=['white_win'])
        samplePos = (data['white_win'] == 'won')*1
    elif(datasetname == 'adult'):
        dataAdult = pd.read_csv("a1a.txt", delim_whitespace=True, header=None, names = ['salery50k','age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country']) 
        if featCART:
            dataAdult = dataAdult[['salery50k','relationship', 'occupation', 'education', 'capital-gain','workclass']]
        data = dataAdult.dropna().sample(n=sampleAmount, random_state=1)
        trainData = data.drop(columns='salery50k')
        samplePos = (data['salery50k'] == 1)*1
        numericalGroup = [1,2,3,4,5,6,7,8,9,10,11,12,13,14]
    elif(datasetname == 'breast'):
        dataBreast = pd.read_csv("breast.txt", delim_whitespace=True, header=0, names=['Class','Sample code number','Clump Thickness','Uniformity of Cell Size','Uniformity of Cell Shape','Marginal Adhesion','Single Epithelial Cell Size','Bare Nuclei','Bland Chromatin','Normal Nucleoli','Mitoses'])
        dataBreast = dataBreast.drop(columns='Sample code number') #drop id-numbers of observations
        if featCART:
            dataBreast = dataBreast[['Class','Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape', 'Bare Nuclei']]
        data = dataBreast.sample(n=sampleAmount, random_state=1)
        trainData = data.drop(columns='Class')
        for i in trainData.columns: #take away the rownumber: of all variables.
                trainData[i] = trainData[i].str[trainData.iloc[0][i].index(':')+1:]
        samplePos = (data['Class'] == 2)*1
        numericalGroup = [1, 2, 3, 4, 5, 6, 7, 8, 9]
    elif(datasetname == 'mushroom'):
        dataMushroom = pd.read_csv("agaricus-lepiota.data", header=0, names=['edible','cap-shape','cap-surface','cap-color','bruises?','odor','gill-attachment','gill-spacing','gill-size','gill-color','stalk-shape','stalk-root','stalk-surface-above-ring','stalk-surface-below-ring','stalk-color-above-ring','stalk-color-below-ring','veil-type','veil-color','ring-number','ring-type','spore-print-color','population','habitat'])
        dataMushroom = dataMushroom.replace("?", np.nan)
        data = dataMushroom.dropna().sample(n=sampleAmount, random_state=1)

        #data = dataMushroom.dropna().sample(n=5, random_state=1).drop(columns=['cap-color','bruises?','odor','gill-attachment','gill-spacing','gill-size','gill-color','stalk-shape','stalk-root','stalk-surface-above-ring','stalk-surface-below-ring','stalk-color-above-ring','stalk-color-below-ring','veil-type','veil-color','ring-number','ring-type','spore-print-color','population','habitat'])

        trainData = data.drop(columns='edible')
        samplePos = (data['edible'] == 'e')*1
    elif(datasetname == 'tictactoe'):
        dataTicTac = pd.read_csv("tic-tac-toe.data", header=None)
        data = dataTicTac.dropna().sample(n=sampleAmount, random_state=1)
        trainData = data.drop(columns=9)
        samplePos = (data[9] == 'positive')*1
    elif(datasetname == 'monks'):
        dataMonks = pd.read_csv("monks-1.test", delim_whitespace=True, header = None)
        data = dataMonks.sample(n=sampleAmount, random_state=1)
        for i in data.columns:
            data[i] = data[i].apply(str)
        trainData = data.drop(columns=[0, 7])
        samplePos = (data[0] == '1')*1
    elif(datasetname == 'votes'):
        dataVotes = pd.read_csv("house-votes-84.data", header=None)
        dataVotes = dataVotes.replace("?", np.nan)
        data = dataVotes.dropna().sample(n=sampleAmount, random_state=1)
        trainData = data.drop(columns=0)
        samplePos = (data[0] == 'democrat')*1
    elif(datasetname == 'heart'):
        dataHeart1 = pd.read_csv("SPECT.test", header=None)
        dataHeart2 = pd.read_csv("SPECT.train", header=None)
        frames = [dataHeart1, dataHeart2]
        dataHeart = pd.concat(frames)
        data = dataHeart.dropna().sample(n=sampleAmount, random_state=1)
        for i in data.columns:
            data[i] = data[i].apply(str)
        trainData = data.drop(columns=0)
        samplePos = (data[0] == '1')*1
    elif(datasetname == 'student'):
        studentMat = pd.read_csv("student-mat.csv", sep=";")
        studentPor = pd.read_csv("student-por.csv", sep=";")
        studentall = pd.merge(studentMat, studentPor, on=["school","sex","age","address","famsize","Pstatus","Medu","Fedu","Mjob","Fjob","reason","nursery","internet"])
        numericalGroup = [31, 30, 29, 28, 3]

    #process data for tree

    start = 0
    for i in trainData.columns:
        if (start == 0):
            trainDataDummiesTemp = pd.get_dummies(trainData[i], prefix=i)
            trainDataDummies = trainDataDummiesTemp[sorted(trainDataDummiesTemp.columns.tolist(), key=str.lower)]
            start = 1
        else:
            trainDataDummiesTemp = pd.get_dummies(trainData[i], prefix=i)
            trainDataDummies = trainDataDummies.join(trainDataDummiesTemp[sorted(trainDataDummiesTemp.columns.tolist(), key=str.lower)])

    groupsN = trainData.shape[1]
    G = range(1, groupsN + 1)
    groupsOfFeatures = trainData.nunique()
    featureToGroup = []
    for g in G:
        amountOfFeatures = groupsOfFeatures.iloc[g-1]
        for f in range(amountOfFeatures):
            featureToGroup.append(g)

In [3]:
dataAdult.nunique()

NameError: name 'dataAdult' is not defined

In [4]:
def decisionTree(typeOfTree, C, ExclSameBranchFollowing, strength, anchor, relax, numerical, combCon, maxCard, sensOn, specOn, sensitivity, specificity):
    model = Model("tree")
    model.parameters.randomseed = 42
    
    #datashape
    N = trainData.shape[0] # amount of samples
    I = range(1, N + 1) # all samples
    groupsN = trainData.shape[1] # amount of groupes
    groupsOfFeatures = trainData.nunique() # amount of features per group
    G = range(1, groupsN + 1) #groups
    J = range(1, groupsOfFeatures.sum() + 1) #features
    Iplus = np.where(samplePos == 1)[0]
    Iplus = [x+1 for x in Iplus]
    Imin = np.where(samplePos == 0)[0]
    Imin = [x+1 for x in Imin]
    
    
    #tree topology
    if (typeOfTree == 1):
        #simple tree of depth 2
        dNodesN = 3
        lNodesN = 4
        Bplus = [1, 3]
        Bmin = [2, 4]
        route = np.array([[1, 1, 0], [1, -1, 0], [-1, 0, 1], [-1, 0, -1]])
        nonAdjLeaf = [1]
        nonAdjLeafSymm = [1]
        AdjLeaf = [2, 3]
    if(typeOfTree == 2):
        #binary tree with depth 2,5
        dNodesN = 5
        lNodesN = 6
        Bplus = [1, 3, 5]
        Bmin = [2, 4, 6]
        route = np.array([[1, 1, 1, 0, 0], [1, 1, -1, 0, 0], [1, -1, 0, 1, 0], \
                          [1, -1, 0, -1, 0], [-1, 0, 0, 0, 1],  [-1, 0, 0, 0, -1]])
        nonAdjLeaf = [1, 2]
        nonAdjLeafSymm = [2]
        AdjLeaf = [3, 4, 5]
    if(typeOfTree == 3):
        #binary tree with depth 3
        dNodesN = 7
        lNodesN = 8
        Bplus = [1, 3, 5, 7]
        Bmin = [2, 4, 6, 8]
        route = np.array([[1, 1, 1, 0, 0, 0, 0], [1, 1, -1, 0, 0, 0, 0], [1, -1, 0, 1, 0, 0, 0], \
                         [1, -1, 0, -1, 0, 0, 0], [-1, 0, 0, 0, 1, 1, 0], [-1, 0, 0, 0, 1, -1, 0], \
                         [-1, 0, 0, 0, -1, 0, 1], [-1, 0, 0, 0, -1, 0, -1]])
        nonAdjLeaf = [1, 2, 5]
        nonAdjLeafSymm = [1, 2, 5]
        AdjLeaf = [3, 4, 6, 7]
    if(typeOfTree == 4):
        #binary tree with depth 3,5
        dNodesN = 7
        lNodesN = 8
        Bplus = [1, 3, 5, 7]
        Bmin = [2, 4, 6, 8]
        route = np.array([[1, 1, 1, 1, 0, 0, 0], [1, 1, 1, -1, 0, 0, 0], [1, 1, -1, 0, 1, 0, 0], \
                         [1, 1, -1, 0, -1, 0, 0], [1, -1, 0, 0, 0, 1, 0], [1, -1, 0, 0, 0, -1, 0], \
                         [-1, 0, 0, 0, 0, 0, 1], [-1, 0, 0, 0, 0, 0, -1]])
        nonAdjLeaf = [1, 2, 3]
        nonAdjLeafSymm = [3]
        AdjLeaf = [4, 5, 6, 7]
        
    #make nodes
    K = range(1, dNodesN + 1) #decision nodes
    B = range(1, lNodesN + 1) #leaf nodes

    # --- decision variables --- 
    idv = [(g, k) for g in G for k in K]
    if(relax): 
        v = model.continuous_var_dict(keys=idv, lb=0, ub=1, name="V") #relaxation of variables of paragraph 4.3.4
    else:
        v = model.binary_var_dict(keys=idv, name="V")
    idz = [(j,k) for j in J for k in K]
    if(relax):
        idzAdjLeaf = [(j,k) for j in J for k in AdjLeaf]
        idzNonAdjLeaf = [(j,k) for j in J for k in nonAdjLeaf]
        z = model.continuous_var_dict(keys=idzAdjLeaf, lb=0, ub=1, name="Z")
        zNonAdjLeaf = model.binary_var_dict(keys=idzNonAdjLeaf, name="Z")
        z.update(zNonAdjLeaf)
    else:
        idz = [(j,k) for j in J for k in K]
        z = model.binary_var_dict(keys=idz, name="Z")
    idc = [(b,i) for b in B for i in I]
    if(relax):
        c = model.continuous_var_dict(keys=idc,ub=1, name="C")
    else:
        c = model.binary_var_dict(keys=idc, name="C")
    idL = [(i,k) for i in I for k in K]
    L = model.binary_var_dict(keys=idL, name="L")
    idR = [(i,k) for i in I for k in K]
    R = model.binary_var_dict(keys=idL, name="R")
    if(numerical):
        idw = [(g,k) for k in K for g in numericalGroup]
        w = model.binary_var_dict(keys=idw, name="W")
    if(combCon):
        if(numerical):
            idw = [(g,k) for k in K for g in G if ((g not in numericalGroup) and (groupsOfFeatures[g-1] > maxCard))]
            wcomb = model.binary_var_dict(keys=idw, name="W")
            w.update(wcomb)
        else:
            idw = [(g,k) for k in K for g in G if ((g not in numericalGroup) and (groupsOfFeatures[g-1] > maxCard))]
            w = model.binary_var_dict(keys=idw, name="W")

    # --- constraints ---
    for k in K:
        model.add_constraint(model.sum(v[g, k] for g in G) == 1) #constraint (1)

    for k in K:
        for j in J:
            model.add_constraint(z[j,k] <= v[featureToGroup[j-1], k]) #constraint (2)

    for k in K:
        for i in I:
            model.add_constraint(L[i,k] == sum(z[j,k] for j in J if trainDataDummies.iloc[i-1,j-1] == 1)) #constraint (3)
            model.add_constraint(R[i,k] == 1 - L[i,k]) #constraint (4)
    
    if(strength):
        for i in I:
            for k in K:
                model.add_constraint(sum(c[b,i] for b in B if route[b-1, k-1] == 1) <= L[i,k]) #constraint (11)
                model.add_constraint(sum(c[b,i] for b in B if route[b-1, k-1] == -1) <= R[i,k]) #constraint (12)
    else:
        for b in B:
            for i in I:
                for k in K:
                    switch = route[b-1, k-1]
                    if switch == 1:
                        model.add_constraint(c[b,i] <= L[i,k]) #constraint (5)
                    elif switch == -1:
                        model.add_constraint(c[b,i] <= R[i,k]) #constraint (6)
        for i in I:
            model.add_constraint(model.sum(c[b,i] for b in B) == 1) #constraint (7)
    
    if(anchor): 
        for g in G:
            for k in nonAdjLeafSymm: #only for nonleafs with symmatrical subtree
                model.add_constraint(z[featureToGroup.index(g) + 1, k] == v[g,k]) #constraint (13)
                        
    if(ExclSameBranchFollowing):
        #for k in K:
        test=1
            #add constraint (9) and (10)
            
    if(numerical):
        #numerical value constraint 4.4
        for k in K:
            for g in numericalGroup:
                featureList = [i for i, x in enumerate(featureToGroup) if x == g]
                for j in featureList:
                    j = j + 1 #making index start at 1
                    if(j-1 == featureList[0]):    
                        model.add_constraint(z[j,k] >= z[j+1,k] - w[g,k])
                    elif(j-1 == featureList[-1]):
                        model.add_constraint(z[j,k] >= z[j-1,k] - (1 - w[g,k]))
                    else:
                        model.add_constraint(z[j,k] >= z[j+1,k] - w[g,k])
                        model.add_constraint(z[j,k] >= z[j-1,k] - (1 - w[g,k]))                     
                
    if(combCon):
        for k in K:
            for g in G:
                if((g not in numericalGroup) and (groupsOfFeatures[g-1] > maxCard)):
                    featureList = [i for i, x in enumerate(featureToGroup) if x == g]
                    model.add_constraint(model.sum(z[j+1,k] for j in featureList) <= maxCard + (groupsOfFeatures[g-1] - maxCard)*(1-w[g,k]))
                    model.add_constraint(model.sum(z[j+1,k] for j in featureList) >= (groupsOfFeatures[g-1] - maxCard) - (groupsOfFeatures[g-1] - maxCard)*w[g,k])
        
        
    
    # --- objective ---
    model.objectiveTruePos = model.sum(model.sum(c[b,i] for b in Bplus) for i in Iplus)
    model.objectiveTrueNeg = model.sum(model.sum(c[b,i] for b in Bmin) for i in Imin)
    
    if(sensOn):
        
        model.add_constraint(model.sum(model.sum(c[b,i] for b in Bplus) for i in Iplus) >= math.ceil((1-sensitivity)*len(Iplus) ))
        
        totalObjective = model.objectiveTrueNeg
        
        sensitivity
    elif(specOn):
        model.add_constraint(model.sum(model.sum(c[b,i] for b in Bmin) for i in Imin) >= math.ceil((1-specificity)*len(Imin) ))

        totalObjective = model.objectiveTruePos
    else:
        totalObjective = model.objectiveTruePos + C*model.objectiveTrueNeg 
    
    model.maximize(totalObjective)
    
    return model

# This is for dataset breast

In [10]:
#--- make model
solTime = 60*60*2 #3 hours
typeOfTree = 3
C = 1  #weight to objective function
ExclSameBranchFollowing = False #constraint (9) and (10)

combCon = False
maxCard = 2
sensOn = False
specOn = False
sensitivity = 0.95
specificity = 0.95


#--- different models
report = list()

strength = True #constraint (11) and (12)
anchor = True #constraint (13)
relax = True

datasetname = 'breast'
featCART = True
sampleAmountLoop = [100, 200, 300, 400, 500, 600]
typeOfTreeLoop = [1, 2, 3, 4]

numerical = False

for sampleAmount in sampleAmountLoop:
    for typeOfTree in typeOfTreeLoop:
        sys.stdout.write('\rStarting with sample ' + str(sampleAmount) + ' with treetype ' + str(typeOfTree))
        report.append('Report of ' + datasetname + ' with samplesize ' + str(sampleAmount) +  ' with treetype ' + str(typeOfTree))
        DataPreperation(datasetname, sampleAmount, featCART)

        model = decisionTree(typeOfTree, C, ExclSameBranchFollowing, strength, anchor, relax, numerical, combCon, maxCard, sensOn, specOn, sensitivity, specificity)
        model.set_time_limit(solTime)

        sol = model.solve()

        report.append('model obj value: ' + str(sol.get_objective_value()))
        report.append('model solve time: ' + str(model.solve_details.time))

        report.append('\n')
        #model.print_information()
        #model.export_as_lp(path="C:\\Users\\Probook")

sys.stdout.write('\rFinished with all models!!!!!!!!!!!!!!!!!!!!!!')

print('\n')
for st in report:
    print(st)

Finished with all models!!!!!!!!!!!!!!!!!!!!!!

Report of breast with samplesize 100 with treetype 1
model obj value: 99.0
model solve time: 0.1559999999590218


Report of breast with samplesize 100 with treetype 2
model obj value: 100.0
model solve time: 0.1559999999590218


Report of breast with samplesize 100 with treetype 3
model obj value: 100.0
model solve time: 0.20300000021234155


Report of breast with samplesize 100 with treetype 4
model obj value: 100.0
model solve time: 0.28199999989010394


Report of breast with samplesize 200 with treetype 1
model obj value: 194.0
model solve time: 0.31299999984912574


Report of breast with samplesize 200 with treetype 2
model obj value: 198.0
model solve time: 8.202999999979511


Report of breast with samplesize 200 with treetype 3
model obj value: 200.0
model solve time: 23.92200000002049


Report of breast with samplesize 200 with treetype 4
model obj value: 200.0
model solve time: 24.656999999890104


Report of breast with samplesize

# This is for dataset chess

In [11]:
#--- make model
solTime = 60*60*2 #2 hours
typeOfTree = 3
C = 1  #weight to objective function
ExclSameBranchFollowing = False #constraint (9) and (10)

combCon = False
maxCard = 2
sensOn = False
specOn = False
sensitivity = 0.95
specificity = 0.95


#--- different models
report = list()

strength = True #constraint (11) and (12)
anchor = True #constraint (13)
relax = True

datasetname = 'chess'
featCART = True
sampleAmountLoop = [100, 200, 300, 400, 500, 600]
typeOfTreeLoop = [1, 2, 3, 4]

numerical = False

for sampleAmount in sampleAmountLoop:
    for typeOfTree in typeOfTreeLoop:
        sys.stdout.write('\rStarting with sample ' + str(sampleAmount) + ' with treetype ' + str(typeOfTree))
        report.append('Report of ' + datasetname + ' with samplesize ' + str(sampleAmount) +  ' with treetype ' + str(typeOfTree))
        DataPreperation(datasetname, sampleAmount, featCART)

        model = decisionTree(typeOfTree, C, ExclSameBranchFollowing, strength, anchor, relax, numerical, combCon, maxCard, sensOn, specOn, sensitivity, specificity)
        model.set_time_limit(solTime)

        sol = model.solve()

        report.append('model obj value: ' + str(sol.get_objective_value()))
        report.append('model solve time: ' + str(model.solve_details.time))

        report.append('\n')
        #model.print_information()
        #model.export_as_lp(path="C:\\Users\\Probook")

sys.stdout.write('\rFinished with all models!!!!!!!!!!!!!!!!!!!!!!')

print('\n')
for st in report:
    print(st)

Finished with all models!!!!!!!!!!!!!!!!!!!!!!

Report of chess with samplesize 100 with treetype 1
model obj value: 87.0
model solve time: 0.1720000000204891


Report of chess with samplesize 100 with treetype 2
model obj value: 94.0
model solve time: 0.5319999998901039


Report of chess with samplesize 100 with treetype 3
model obj value: 95.0
model solve time: 0.9839999999385327


Report of chess with samplesize 100 with treetype 4
model obj value: 96.0
model solve time: 3.827999999979511


Report of chess with samplesize 200 with treetype 1
model obj value: 175.0
model solve time: 0.3279999999795109


Report of chess with samplesize 200 with treetype 2
model obj value: 186.0
model solve time: 0.75


Report of chess with samplesize 200 with treetype 3
model obj value: 186.0
model solve time: 1.031999999890104


Report of chess with samplesize 200 with treetype 4
model obj value: 191.0
model solve time: 5.952999999979511


Report of chess with samplesize 300 with treetype 1
model obj

# This is for the adult dataset

In [5]:
#--- make model
solTime = 60*60*2 #2 hours
C = 1  #weight to objective function
ExclSameBranchFollowing = False #constraint (9) and (10)

combCon = False
maxCard = 2
sensOn = False
specOn = False
sensitivity = 0.95
specificity = 0.95


#--- different models
report = list()

strength = True #constraint (11) and (12)
anchor = True #constraint (13)
relax = True

datasetname = 'adult'
featCART = True
sampleAmountLoop = [100, 200, 300, 400, 500, 600]
typeOfTreeLoop = [1, 2, 3, 4]

numerical = False

for sampleAmount in sampleAmountLoop:
    for typeOfTree in typeOfTreeLoop:
        sys.stdout.write('\rStarting with sample ' + str(sampleAmount) + ' with treetype ' + str(typeOfTree))
        report.append('Report of ' + datasetname + ' with samplesize ' + str(sampleAmount) +  ' with treetype ' + str(typeOfTree))
        DataPreperation(datasetname, sampleAmount, featCART)

        model = decisionTree(typeOfTree, C, ExclSameBranchFollowing, strength, anchor, relax, numerical, combCon, maxCard, sensOn, specOn, sensitivity, specificity)
        model.set_time_limit(solTime)

        sol = model.solve()

        report.append('model obj value: ' + str(sol.get_objective_value()))
        report.append('model solve time: ' + str(model.solve_details.time))

        report.append('\n')
        #model.print_information()
        #model.export_as_lp(path="C:\\Users\\Probook")

sys.stdout.write('\rFinished with all models!!!!!!!!!!!!!!!!!!!!!!')

print('\n')
for st in report:
    print(st)

Finished with all models!!!!!!!!!!!!!!!!!!!!!!

Report of adult with samplesize 100 with treetype 1
model obj value: 92.0
model solve time: 0.2029999999795109


Report of adult with samplesize 100 with treetype 2
model obj value: 94.0
model solve time: 5.968000000109896


Report of adult with samplesize 100 with treetype 3
model obj value: 96.0
model solve time: 25.32800000021234


Report of adult with samplesize 100 with treetype 4
model obj value: 95.0
model solve time: 103.29700000002049


Report of adult with samplesize 200 with treetype 1
model obj value: 175.0
model solve time: 0.6400000001303852


Report of adult with samplesize 200 with treetype 2
model obj value: 180.0
model solve time: 59.26599999982864


Report of adult with samplesize 200 with treetype 3
model obj value: 184.0
model solve time: 161.46900000004098


Report of adult with samplesize 200 with treetype 4
model obj value: 183.0
model solve time: 684.3430000001099


Report of adult with samplesize 300 with treetyp